In [171]:
pwd

'/home/harshal/Documents/Projects/GE/speech_to_text/deepspeech-0.6.1-models'

In [ ]:
#Requirements
deepspeech
wave
json
numpy
scipy

In [3]:
cd deepspeech-0.6.1-models/

/home/harshal/Documents/Projects/GE/speech_to_text/deepspeech-0.6.1-models


In [236]:
from deepspeech import Model
import numpy as np
import speech_recognition as sr
import wave
import json

In [ ]:
sample_rate = 16000
BEAM_WIDTH = 500
LM_ALPHA = 0.75
LM_BETA = 1.85

In [237]:
def words_from_metadata(metadata):
    word = ""
    word_list = {}
    word_start_time = 0
    # Loop through each character
    for i in range(0, metadata.num_items):
        item = metadata.items[i]
        # Append character to word if it's not a space
        if item.character != " ":
            if len(word) == 0:
                # Log the start time of the new word
                word_start_time = item.start_time

            word = word + item.character
        # Word boundary is either a space or the last character in the array
        if item.character == " " or i == metadata.num_items - 1:
            word_duration = item.start_time - word_start_time

            if word_duration < 0:
                word_duration = 0

            
            each_word = {}
            each_word['start_time'] = round(word_start_time, 4)
            each_word['duration'] = round(word_duration, 4)
            
#             each_word["start_time"] = {round(word_start_time, 4)}
#             each_word["duration"] = round(word_duration, 4)
            if word not in word_list.keys():
                word_list[word]=[]
            word_list[word].append(each_word)
            # Reset
            word = ""
            word_start_time = 0

    return word_list

In [312]:
dirPath = "/home/harshal/Documents/Projects/GE/speech_to_text/"
pathToAudioFile = "audio/test_2.wav"
newpath = "audio/test_2_deid.wav"

model_name = dirPath + "deepspeech-0.6.1-models/output_graph.pbmm"
langauage_model = dirPath + "deepspeech-0.6.1-models/lm.binary"
trie = dirPath + "deepspeech-0.6.1-models/trie"
audio_file = dirPath + pathToAudioFile
audio_deid = dirPath + newpath

if __name__ == '__main__':
    ds = Model(model_name, BEAM_WIDTH)
    ds.enableDecoderWithLM(langauage_model, trie, LM_ALPHA, LM_BETA)
    
    #Read original audio file
    file_obj = open(audio_file,'rb')
    fin = wave.open(file_obj)
    fs = fin.getframerate()
    print("Framerate: ", fs)

    audio = np.frombuffer(fin.readframes(fin.getnframes()), np.int16)

    audio_length = fin.getnframes() * (1/sample_rate)
    timestamp = words_from_metadata(ds.sttWithMetadata(audio))
    fin.close()
    file_obj.close()

    print("Infering {} file".format(audio_file))
    transcript = ds.stt(audio)
    print(transcript)

Framerate:  16000
Infering /home/harshal/Documents/Projects/GE/speech_to_text/audio/test_2.wav file
a hand was isa


In [306]:
audio_length

1.2974166666666667

In [307]:
timestamp

{'i': [{'start_time': 0.0, 'duration': 0.54}],
 'was': [{'start_time': 0.66, 'duration': 1.64}],
 'a': [{'start_time': 2.48, 'duration': 0.12},
  {'start_time': 4.56, 'duration': 0.66},
  {'start_time': 6.52, 'duration': 0.0}],
 'lad': [{'start_time': 2.64, 'duration': 1.62}],
 'is': [{'start_time': 5.34, 'duration': 0.9}]}

In [308]:
def get_deidentified_file(input_audio_file_path, timestamp, phi_word_list, fs=16000):
    #Read file as a numpy array
    from scipy.io.wavfile import read
    a = read(input_audio_file_path)
    arr = np.array(a[1],dtype=np.int16)
    #Each word in given list
    for word in phi_word_list:
        #Each instance of the word
        for i in range(len(timestamp[word])):
            #Get start position in array
            start = int(timestamp[word][i]['start_time']*fs)
            #Get end position in array
            end = int((timestamp[word][i]['duration']*fs) - start)
            #Mute the required part
            arr[start: end] = 0
    return arr

In [310]:
array = get_deidentified_file(audio_file, timestamp=timestamp, phi_word_list=['a'])

In [311]:
#Write in new audio file
from scipy.io.wavfile import write
write(audio_deid, rate=16000, data=array)

In [133]:
truecasing_by_pos(transcript)

'Why should one Halt on the Way'

In [132]:
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import re
def truecasing_by_pos(input_text):
    # tokenize the text into words
    words = nltk.word_tokenize(input_text)
    # apply POS-tagging on words
    tagged_words = nltk.pos_tag([word.lower() for word in words])
    # apply capitalization based on POS tags
    capitalized_words = [w.capitalize() if t in ["NN","NNS"] else w for (w,t) in tagged_words]
    # capitalize first word in sentence
    capitalized_words[0] = capitalized_words[0].capitalize()
    # join capitalized words
    text_truecase = re.sub(" (?=[\.,'!?:;])", "", ' '.join(capitalized_words))
    return text_truecase

In [13]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/harshal/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True